# Exemple d'appel à l'API prédicat

Created on Tue Jan 19 09:29:03 2021

@author: Yves-Laurent Benichou et Jérémy L'Hour

In [24]:
import requests
import pandas as pd

In [25]:
########## DEFINITION DES FONCTIONS PRINCIPALES ##########

def predict_using_api(libelle, session, k=1, trace=False):
    """
    predict_using_api:
        get the top-k prediction for a given libelle
    
    @param libelle (list): list of str for which to predict label 
    @param session (): a requests.sesssion() object
    @param k (int): top-k
    
    @return: a json of the results
    """
    call_url = 'https://api.lab.sspcloud.fr/predicat/label?q='+'&q='.join(libelle)+'&k='+str(k)
    response = session.get(call_url)
    if response.ok and trace:
        print('Prediction received.')
    return response.json()

def decode_na2008(categorie, session):
    """
    decode_na2008:
        decode the four character of NA2008
    
    @param categorie (str):
    @param session (): a requests.sesssion() object
    """
    call_url = 'https://api-predicat.kub.sspcloud.fr/na2008?q='+'&q='.join(categorie)
    response = session.get(call_url)
    return response.json()

# Ouverture d'une session, pour communiquer sur le même numéro de port à chaque requête
s = requests.session()

In [26]:
# Une seule prédiction pour un libelle

libelle = 'Compote de Pommes'
prediction = predict_using_api([libelle], session=s, trace=True)
print(prediction)

Prediction received.
{'Compote de Pommes': [{'label': 'C10C', 'proba': '1.000', 'confiance': '0.998'}]}


In [27]:
# Retrouver l'intitulé de la categorie
label = prediction[libelle][0]['label']
dico = decode_na2008([label], session=s)
print('Le code '+label+' correspond à '+dico[label])

Le code C10C correspond à Transformation et conservation de fruits et légumes


In [28]:
# Plusieurs prédictions pour un libelle

libelle = 'Compote de Pommes'
predictions = predict_using_api([libelle], session=s, k=5, trace=True)
print(predictions)

Prediction received.
{'Compote de Pommes': [{'label': 'C10C', 'proba': '1.000', 'confiance': '0.998'}, {'label': 'C10B', 'proba': '0.001', 'confiance': None}, {'label': 'A01Z', 'proba': '0.001', 'confiance': None}, {'label': 'C15Z', 'proba': '0.000', 'confiance': None}, {'label': 'C12Z', 'proba': '0.000', 'confiance': None}]}


In [29]:
# Affichage du top-k
for i, item in enumerate(predictions[libelle]):
    print('Prédiction '+str(i+1)+': '+item['label']+' avec probabilité '+item['proba'])

Prédiction 1: C10C avec probabilité 1.000
Prédiction 2: C10B avec probabilité 0.001
Prédiction 3: A01Z avec probabilité 0.001
Prédiction 4: C15Z avec probabilité 0.000
Prédiction 5: C12Z avec probabilité 0.000


In [30]:
# Une prédiction pour plusieurs libelles et mise dans un pd.DataFrame()

libelles = ['Compote de pommes', 'Yaourts 0% x6', 'Biscuits chocolat x2']

predictions = predict_using_api(libelles, session=s)
row_list = [predictions[item][0] for item in libelles]

df = pd.DataFrame(row_list)
df['libelle'] = libelles
df.head()

,label,proba,confiance,libelle
0,C10C,1.000,0.998,Compote de pommes
1,C10E,0.938,0.546,Yaourts 0% x6
2,C10G,1.000,0.995,Biscuits chocolat x2


In [31]:
# A partir d'un csv

def predict_from_csv(file, session):
    """
    predict_from_csv:
        inputs a csv file and outputs the same file with three extra columns (label, proba, confiance)
        
    @param file (str): a csv file
    @param session (): a requests.session() object
    """
    with open(file) as f:
        df = pd.read_csv(f)
        
    predictions = predict_using_api(df['libelle'].to_list(), session=session)
    row_list = [predictions[item][0] for item in df['libelle'].to_list()]

    final_df = pd.DataFrame(row_list)
    final_df['libelle'] = df['libelle']
    final_df = final_df[['libelle', 'label', 'proba', 'confiance']]
    final_df.to_csv('output.csv')
    return final_df


In [32]:
final_df = predict_from_csv('libelles.csv', session=s)
final_df.head()

,libelle,label,proba,confiance
0,Compote abricot,C10C,0.986,0.842
1,Yaourt Fruits Yoplait,C10E,1.000,1.000
2,Jus d'orange,C11Z,0.749,0.327
3,Oranges 500g,A01Z,0.492,0.295
4,Gnocchi Lustucru,C10C,0.995,0.979


In [34]:
dico = decode_na2008(final_df['label'].to_list(), session=s)
final_df['intitule'] = final_df['label'].map(dico)
final_df.head(20)

,libelle,label,proba,confiance,intitule
0,Compote abricot,C10C,0.986,0.842,Transformation et conservation de fruits et lé...
1,Yaourt Fruits Yoplait,C10E,1.000,1.000,Fabrication de produits laitiers
2,Jus d'orange,C11Z,0.749,0.327,Fabrication de boissons
3,Oranges 500g,A01Z,0.492,0.295,"Culture et production animale, chasse et servi..."
4,Gnocchi Lustucru,C10C,0.995,0.979,Transformation et conservation de fruits et lé...
5,T-Shirt HOMME XXL,C14Z,0.993,0.985,Industrie de l'habillement
6,Robe ete taille M,C14Z,0.988,0.987,Industrie de l'habillement
7,Baskets 41 H,C15Z,1.000,1.000,Industrie du cuir et de la chaussure
8,Pates barilla,C10G,1.000,1.000,Fabrication de produits de boulangerie-pâtisse...
9,biberon tetine silicone,C22B,0.995,0.922,Fabrication de produits en plastique
